# Web Elicitation Data

## import module

In [7]:
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import os
import requests
import models
import pandas as pd
import datetime

#### access to environment variable

In [8]:
load_dotenv()

True

## get api token for request to https://openweathermap.org/

In [9]:
api_key = os.environ.get('API_KEY')

#### selected data with specific Latitude and Longitude

In [10]:
lat_long = [(51.507351, -0.127758), (53.408371, -2.991573), (40.712776, -74.005974), (41.874316, -87.631724),
            (38.893883, -77.044142), (43.686677, -79.392166), (49.243843, -123.112445), (35.689133, 51.389524),
            (36.470552, 52.348881), (36.545611, 52.684186), (48.136583, 11.572022), (49.461167, 11.071938)]


### function for get data

In [11]:
def request_api(lat_long_var):
    """
    a tuple that carries Latitude and Longitude:param lat_long_var:tuple
    a json that carries all data like weather , city , etc.:return: json
    """
    return requests.get(
        f'https://api.openweathermap.org/data/2.5/forecast?lat={lat_long_var[0]}&lon={lat_long_var[1]}&appid={api_key}')


#### connection to data base

In [12]:
engine = create_engine(os.environ.get("DATABASE_URL"))

#### interaction with database

In [13]:
Session = sessionmaker(bind=engine)
session = Session()

#### this function registers weather data and city data
####   in database and Builds a relationship with city and weather

In [14]:
def register_weather_data(data):
    """
    this function registers weather data and city data
    in database and Builds a relationship with city and weather

    all data we are getting from open weather site:param data:json
    message:return:string
    """
    city = models.City(id=data['city']['id'], city_name=data['city']['name'],
                       country=data['city']['country'],
                       coord_lat=data['city']['coord']['lat'], coord_lon=data['city']['coord']['lon'])
    """register data in data base"""
    condition = session.query(models.City).filter_by(id=data['city']['id']).first()
    if not condition:
        session.add(city)
        session.commit()
    else:
        return 'this data be Register in past'

    for forecast in data['list']:
        weather = models.Weather(date=forecast['dt_txt'][0:10], time=forecast['dt_txt'][11:19],
                                 temperature=forecast['main']['temp'],
                                 humidity=forecast['main']['humidity'],
                                 wind_speed=forecast['wind']['speed'], city_id=city.id)
        """register data in data base"""
        session.add(weather)
        session.commit()
    """close connection"""
    session.close()
    return 'data registered successfully'


### practical of project

In [15]:
for lat_long_step in lat_long:
    while True:
        response = request_api(lat_long_step)
        if response.status_code == 200:
            result = response.json()
            print(register_weather_data(result))
            break
print('done')

this data be Register in past
this data be Register in past
this data be Register in past
this data be Register in past
this data be Register in past
this data be Register in past
this data be Register in past
this data be Register in past
this data be Register in past
this data be Register in past
this data be Register in past
this data be Register in past
done


# pandas

fetching cities data from data base

In [16]:
cities = session.query(models.City).all()

In [17]:
df = pd.DataFrame({
                          'city_name': city.city_name,
                          'country': city.country,
                          'coord_lat': city.coord_lat,
                          'coord_lon': city.coord_lon,
                      } for city in cities
                      )
print(df)

          city_name country  coord_lat  coord_lon
0            London      GB    51.5074    -0.1278
1         Liverpool      GB    53.4084    -2.9916
2          New York      US    40.7128   -74.0060
3           Chicago      US    41.8743   -87.6317
4   Washington D.C.      US    38.8939   -77.0441
5           Toronto      CA    43.6867   -79.3922
6         Vancouver      CA    49.2438  -123.1124
7            Tehran      IR    35.6891    51.3895
8              Āmol      IR    36.4706    52.3489
9             Babol      IR    36.5456    52.6842
10         Altstadt      DE    48.1366    11.5720
11        Nuremberg      DE    49.4612    11.0719


### fetching cities data from data base

In [19]:
weather_info = session.query(models.Weather).all()


this cacher for weekly info

In [21]:
all_weather_every_city=[]

# 24h data

In [22]:
    for city in cities:
        df_1 = pd.DataFrame({
                                'date': w.date,
                                'time': w.time,
                                'temperature': w.temperature,
                                'humidity': w.humidity,
                                'wind_speed': w.wind_speed,
                                'city_name': city.city_name
                            } for w in weather_info
                            )
        """ 
        with this query we can define want 24h data of every city
        select_24h is only int data 
        """
        try:
            select_24h = session.query(models.Weather).filter_by(date=str(datetime.date.today())).filter_by(
                city_id=session.query(models.City).all()[0].id).all()
        except ValueError:
            print('in register city we have problem')
            select_24h=0

        print(df_1.head(len(select_24h)+1))
        # with this code caching every cities data that fetching from database
        all_weather_every_city.append(df_1)

         date      time  temperature  humidity  wind_speed city_name
0  2024-10-02  03:00:00       286.25      91.0        4.24    London
1  2024-10-02  06:00:00       286.22      93.0        2.93    London
2  2024-10-02  09:00:00       287.22      84.0        4.80    London
3  2024-10-02  12:00:00       287.92      79.0        3.75    London
4  2024-10-02  15:00:00       288.38      74.0        4.70    London
5  2024-10-02  18:00:00       286.39      77.0        5.32    London
6  2024-10-02  21:00:00       285.70      70.0        4.77    London
7  2024-10-03  00:00:00       285.42      65.0        4.29    London
         date      time  temperature  humidity  wind_speed  city_name
0  2024-10-02  03:00:00       286.25      91.0        4.24  Liverpool
1  2024-10-02  06:00:00       286.22      93.0        2.93  Liverpool
2  2024-10-02  09:00:00       287.22      84.0        4.80  Liverpool
3  2024-10-02  12:00:00       287.92      79.0        3.75  Liverpool
4  2024-10-02  15:00:00      

## represent to us all weather information in database for every city

In [25]:
    for w_1 in all_weather_every_city:
        print(w_1)

           date      time  temperature  humidity  wind_speed city_name
0    2024-10-02  03:00:00       286.25      91.0        4.24    London
1    2024-10-02  06:00:00       286.22      93.0        2.93    London
2    2024-10-02  09:00:00       287.22      84.0        4.80    London
3    2024-10-02  12:00:00       287.92      79.0        3.75    London
4    2024-10-02  15:00:00       288.38      74.0        4.70    London
..          ...       ...          ...       ...         ...       ...
475  2024-10-06  12:00:00       287.59      56.0        3.23    London
476  2024-10-06  15:00:00       286.33      63.0        3.03    London
477  2024-10-06  18:00:00       282.43      79.0        1.63    London
478  2024-10-06  21:00:00       282.34      80.0        1.94    London
479  2024-10-07  00:00:00       280.59      90.0        2.11    London

[480 rows x 6 columns]
           date      time  temperature  humidity  wind_speed  city_name
0    2024-10-02  03:00:00       286.25      91.0    